In [ ]:
!pip install -q transformers

In [ ]:
!pip install -q torch torchvision

In [ ]:
!pip install -q pillow

In [ ]:
from transformers import DetrImageProcessor, DetrForObjectDetection
import torch
from PIL import Image
import os
from collections import Counter

In [ ]:
def process_images(base_folder_path, categories):
    """
    Process images in specified categories and generate detection results and object frequency analysis.

    Args:
    - base_folder_path (str): Base folder path containing image categories.
    - categories (list): List of categories to process.

    Returns:
    - output_file_paths (list): List containing paths of the output files generated.
    """

    # Initialize DETR model and image processor
    processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50", revision="no_timm")
    model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50", revision="no_timm")

    output_file_paths = []  # List to store paths of output files generated

    for category in categories:
        current_folder_path = os.path.join(base_folder_path, category)

        # Define output file paths
        output_file_path = f"output_results_{category}.txt"
        frequency_file_path = f"object_frequency_{category}.txt"

        # Counter to keep track of object frequencies
        object_counter = Counter()

        with open(output_file_path, "w") as output_file:
            # Iterate through images in current category
            for img_path in os.listdir(current_folder_path):
                full_img_path = os.path.join(current_folder_path, img_path)

                # Open and preprocess image
                image = Image.open(full_img_path).convert("RGB")
                inputs = processor(images=image, return_tensors="pt", input_data_format="channels_last")

                # Perform inference
                outputs = model(**inputs)

                # Convert outputs to COCO API format with threshold > 0.9
                target_sizes = torch.tensor([image.size[::-1]])
                results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]

                # Write image path to output file
                output_file.write(f"Image - {full_img_path} :\n")

                # Write detection results into output file and update counter
                for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
                    detected_object = model.config.id2label[label.item()]
                    object_counter[detected_object] += 1
                    box = [round(i, 2) for i in box.tolist()]
                    output_file.write(f"Detected {detected_object} with confidence {round(score.item(), 3)} at location {box}\n")

                output_file.write("\n")

        # Write object frequency analysis to separate file
        with open(frequency_file_path, "w") as freq_file:
            for object, count in object_counter.most_common():
                freq_file.write(f"{object}: {count}\n")

        output_file_paths.extend([output_file_path, frequency_file_path])  # Add output file paths to the list

        print(f"Detection results and object frequency analysis for {category} saved.")

    return output_file_paths  # Return list of output file paths

In [ ]:
# Define base folder path containing image categories
base_folder_path = '/Users/aditya/Desktop/task/hateful_memes/img'

In [ ]:
# Define classifying categories
categories = ["hateful_memes", "not_hateful_memes"]

In [ ]:
# Process images and get output file paths
output_paths = process_images(base_folder_path, categories)

In [ ]:
# Print paths of the output files generated
print("Output file paths:")
for path in output_paths:
    print(path)